In [ ]:
pip install kafka-python

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import date,timedelta

In [ ]:
import pandas as pd
from kafka import KafkaProducer
from kafka.admin import KafkaAdminClient, NewTopic
from time import sleep
from json import dumps
import json
import bs4 as bs
import requests
import pickle
import re

# Get all tickers from sp500

In [ ]:
resp = requests.get('https://stockmarketmba.com/stocksinthesp500.php')
soup = bs.BeautifulSoup(resp.text, 'lxml')

In [ ]:
table = soup.find('table', {'id':'ipos'})
print(table)
tickers = []
for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    if len(ticker)>1:
        tickers.append(ticker)

In [ ]:
tickers

In [ ]:
with open("tickers.pickle","wb") as f:
    pickle.dump(tickers,f)
#split of the new line <tag>
tickers_result = []
for i in tickers:
    tickers_result.append(i.split('\n')[0])

In [ ]:
tickers_result

In [ ]:
admin_client = KafkaAdminClient(
    bootstrap_servers="54.166.223.82:9092", 
    client_id='client_1'
)

In [ ]:
topic = NewTopic(name="stock_market_analysis", num_partitions=1, replication_factor=1)
admin_client.create_topics(new_topics=[topic], validate_only=False)

In [11]:
producer = KafkaProducer(
                bootstrap_servers=['54.166.223.82:9092'],
                value_serializer=lambda x: 
                dumps(x).encode('utf-8')
)

In [ ]:
# tickers
# indices = ['NFLX','MSFT','AAPL','GOOGL','SNOW','TSLA','AMZN','META']

In [32]:
def get_daily_stock_data(indices):
    yesterday =  str(date.today() + timedelta(days = -1))
    today =  str(date.today())
    stock_data = pd.DataFrame()
    for index in indices:
        data = yf.download(tickers = index, start = yesterday, end = today)
        data = data.assign(stock_index = index)
        data = data.assign(date = data.index)
        stock_data = pd.concat([stock_data, data])
    return stock_data.to_json(orient = 'records'),stock_data

In [33]:
stock_data, st = get_daily_stock_data(tickers)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [34]:
st

,Open,High,Low,Close,Adj Close,Volume,stock_index,date
Date,,,,,,,,
2023-01-03,12.910000,13.000000,12.530000,12.740000,12.740000,21865100.0,AAL,2023-01-03
2023-01-04,12.860000,13.610000,12.860000,13.590000,13.590000,34333400.0,AAL,2023-01-04
2023-01-03,146.160004,151.660004,146.000000,151.539993,151.539993,1307400.0,AAP,2023-01-03
2023-01-04,152.729996,154.500000,150.889999,151.889999,151.889999,834200.0,AAP,2023-01-04
2023-01-03,130.279999,130.899994,124.169998,125.070000,125.070000,112117500.0,AAPL,2023-01-03
...,...,...,...,...,...,...,...,...
2023-01-04,263.619995,265.809998,258.230011,262.989990,262.989990,311600.0,ZBRA,2023-01-04
2023-01-03,48.680000,49.799999,48.259998,48.459999,48.459999,1175800.0,ZION,2023-01-03
2023-01-04,49.080002,50.840000,48.970001,50.230000,50.230000,1689000.0,ZION,2023-01-04


In [37]:
producer.send('stock_market_analysis', value = stock_data)

#  Load historical Data from Jan 2021 to Dec 2022

In [16]:
stock_data = pd.DataFrame()
for index in tickers_result:
    data = yf.download(tickers = index, start = '2022-12-31' , end = '2023-01-03')
    data = data.assign(stock_index = index)
    data = data.assign(date = data.index)
    stock_data = pd.concat([stock_data, data])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [18]:
stock_data['date'].max()
# stock_data.to_csv('~/Documents/stock_history.csv')

Timestamp('2022-12-30 00:00:00')

In [ ]:
producer.send('stock_market_analysis', value = stock_data.to_json(orient = 'records'))